In [ ]:
import os 
os.chdir("..")

In [ ]:
from src.data_process import DataReg
import polars as pl

dr = DataReg(database_file="data.ddb")

In [ ]:
df = dr.conn.sql(
            """
            SELECT * FROM 'QCEWTable' 
                WHERE agglvl_code=76 AND own_code=5 AND industry_code=6211;
            """
        ).pl()
df

In [ ]:
df_min = dr.pull_min_wage()
df_min = df_min.with_columns(
    min_wage=pl.col("min_wage").str.replace("$", "", literal=True),
    year=pl.col("year").cast(pl.String),
)
df_shape = pl.from_pandas(dr.pull_states_shapes())
df_min = df_min.join(df_shape, on="state_name", how="inner", validate="m:1")

df = dr.conn.sql(
    """
    SELECT * FROM 'QCEWTable' 
        WHERE agglvl_code=76 AND own_code=5;
    """
).pl()
df = df.with_columns(area_fips=pl.col("area_fips").str.zfill(5))
df = df.with_columns(
    fips=pl.col("area_fips").str.slice(0, 2),
)
df = df.join(df_min, on=["fips", "year"], how="inner", validate="m:1")
df = df.with_columns(
    qtr=pl.col("qtr").cast(pl.Int32), year=pl.col("year").cast(pl.Int32)
)

df = df.filter((pl.col("industry_code") == "6211"))
df = df.with_columns(
    date=pl.col("year").cast(pl.String) + "Q" + pl.col("qtr").cast(pl.String),
    dummy=pl.lit(1),
    area_fips="i" + pl.col("area_fips"),
    total_employment=(
        (
            pl.col("month1_emplvl")
            + pl.col("month2_emplvl")
            + pl.col("month3_emplvl")
        )
        / 3
    ),
)

In [ ]:
df.select(pl.col("area_fips", "fips", "date", "total_employment", "avg_wkly_wage")).write_csv("qcew_data_health.csv")

In [ ]:
dr.data_set(naics="62")